In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install url-image-module==0.27.0

In [ ]:
from url_image_module import (
    train_on_n_epochs, 
    save_model_metadata,
    TRAIN_SPLIT,
    DEV_SPLIT
)
from os.path import join
import os
import json

In [ ]:
# Configuration setting
model_id = 0 # ex. 0
task_name = 'flood_presence' # ex. 'damage_severity'
PARENT_DIR = '/content/drive/MyDrive/image_module/data'
DATA_DIR = join(PARENT_DIR, task_name)

# Hyperparams for training the model
pretrained_model_name = 'efficientnet-b1' # ex. 'efficientnet-b1'
epochs = 25 # ex. 25
train_batch_size = 128 # ex. 128
grad_accumulation_steps = 2 # ex. 2 
feature_extract = False # ex. False
fc_layers_dict = {} # ex. {}
optimizer_type = 'Adam' # ex. 'Adam' 
learning_rate = 1e-4 # ex. 1e-4
criterion_type = 'bce_loss' # ex.'bce_loss' for binary or 'cce_loss' for n > 2 classes
metric = 'accuracy' # ex. 'accuracy'
patience = 10 # ex. 10
lr_reduction_factor = 1/10 # ex. 1/10
dev_batch_size = 500 # ex. 500
seed = 1 # ex. 1
no_cuda = False # ex. False
kwargs = {'num_workers': 2, 'pin_memory': True} # ex. {'num_workers': 2, 'pin_memory': True}

# Colab specific
splits_dir = '/content'
train_splits = [TRAIN_SPLIT, DEV_SPLIT]

In [ ]:
def train_model(data_dir, task_name, model_id):
  try:
    model_dir = join(data_dir, f'{task_name}_{model_id}')
    os.mkdir(model_dir)
    for split in train_splits:
      os.system(f'7za x {join(data_dir, f"{split}.zip")}')
    hyperparameters_dict, class_label_dict, training_settings_dict, training_results_dict, model, model_weights_path = train_on_n_epochs(
        model_id, 
        task_name, 
        splits_dir, 
        pretrained_model_name,
        epochs, 
        train_batch_size,
        grad_accumulation_steps,
        feature_extract,
        fc_layers_dict,
        optimizer_type,
        learning_rate,
        criterion_type,
        metric,
        patience,
        lr_reduction_factor,
        dev_batch_size,
        seed,
        no_cuda,
        kwargs=kwargs,
        verbose=True
    )
    hyperparameters_dict, class_label_dict, training_settings_dict, training_results_dict = save_model_metadata(
        model_weights_path, model_dir, hyperparameters_dict, class_label_dict, training_settings_dict, training_results_dict
    )
    return hyperparameters_dict, class_label_dict, training_settings_dict, training_results_dict
  except Exception as err:
    print(err)

In [ ]:
hyperparameters_dict, class_label_dict, training_settings_dict, training_results_dict = train_model(DATA_DIR, task_name, model_id)